In [ ]:
import numpy as np
import scipy.constants
import matplotlib.pyplot as plt

from tqdm import tqdm

import os
import subprocess

# Load Data

In [ ]:
def cgs_to_geo(data): # these are SI-ish geometric units, not cgs-ish geometric units
    data_geo = np.copy(data)
    data_geo[:,:,0] = data[:,:,0] * 1E-2 # radius [m]
    data_geo[:,:,1] = data[:,:,1] * 1E-2 / scipy.constants.c # velocity [cm/s] -> velocity []
    data_geo[:,:,2] = data[:,:,2] * 1E-3 * ( scipy.constants.G / scipy.constants.c**2 ) # grav. mass [g] -> grav. mass [m]
    data_geo[:,:,3] = data[:,:,3] * (1E-3 / (1E-2)**3 ) * ( scipy.constants.G / scipy.constants.c**2 ) # density [g/cm^3] -> density [1/m^2] 
    return data_geo

def load_data(run_dir):
    N_zones = 180
    
    data_dir = os.path.join(run_dir, "data")
    stp_paths = [
        os.path.join(data_dir, d) 
        for d in os.listdir(data_dir) 
        if "stp" in d
    ]

    stp_paths.sort()

    N_stp = len(stp_paths)
    
    data = np.zeros(( N_stp, N_zones, 6 ))

    times = np.zeros(N_stp)

    for i in tqdm(range(N_stp)):
        stp_path = stp_paths[i]

        time_str = subprocess.check_output(
            "grep 'time' %s" % stp_path, shell=True
        ).decode("utf-8")

        times[i] = float(time_str[ time_str.index("time:")+6: ])

        stp_data = np.genfromtxt(stp_path, skip_header=10, skip_footer=180)

        # radius [cm]
        data[i,:,0] = stp_data[:,2]

        # velocity [cm/s]
        data[i,:,1] = stp_data[:,3]

        # grav. mass [g]
        data[i,:,2] = stp_data[:,4]

        #for j in range(1,N_zones):
        #    data[i,j,2] -= np.sum( stp_data[0:j,4] )

        # rest mass density [g/cm^3]
        data[i,:,3] = stp_data[:,5]

        # lapse
        data[i,:,4] = stp_data[:,8]
                        
    # load bounce time
    with open( os.path.join(run_dir, "bounce_time.d") ) as bounce_file:
        bounce_time = float( bounce_file.read() )
        
    data_geo = cgs_to_geo(data)
    
    return data, data_geo, times, bounce_time

In [ ]:
#run_dir = "./s15.0_trise400_k4.3"
exp_data, exp_data_geo, exp_times, exp_bounce_time = load_data("./s15.0_trise400_k4.3")
bh_data,  bh_data_geo,  bh_times,  bh_bounce_time  = load_data("./s22.2_trise400_k4.3")

## Calculate $\Gamma$
Note, this is not the adiabatic index.
In geometric units, from eqn. (7.7) of Liebendorfer's Thesis,
$$\Gamma = \sqrt{ 1 + u^2 - 2m /r }$$
where $u$ is the velocity, $r$ is the radius, and $m$ the gravitational mass.

In [ ]:
def agile_gamma(mass_geo, velocity_geo, radius_geo):
    """
    Calculates the Gamma term defined in equation (7.7), Appendix 7 of 
    Liebendorfer's thesis (https://ui.adsabs.harvard.edu/abs/2000PhDT.......311L/abstract).
    
    Arguments
    =========
    mass_geo: float or float ndarray, gravitational mass in geometric units,
        in a single zone or all zones we want to calculate Gamma at.
    velocity_geo: float or float ndarray, velocity term u in geometric units (defined in 
        Appendix 7 of Liebendorfer's thesis; %au of the state_vector type in PUSH.)
    radius_geo: float or float ndarray, radius in geometric units
    
    Returns
    =======
    float or float ndarray: Gamma in the single zone or list of zones that we so desire
        to know it at.
    """
    
    return np.sqrt(
        1 + velocity_geo**2 - (2 * mass_geo) / radius_geo
    )
    
exp_data_geo[:,1:,5] = agile_gamma(
    exp_data_geo[:,1:,2], exp_data_geo[:,1:,1], exp_data_geo[:,1:,0]
)

bh_data_geo[:,1:,5] = agile_gamma(
    bh_data_geo[:,1:,2], bh_data_geo[:,1:,1], bh_data_geo[:,1:,0]
)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8),dpi=300)

stp_idx = -1

ax.plot( np.log10(bh_data[stp_idx,1:,0]), bh_data[stp_idx,1:,4] )
ax.set_xlabel(r"$\log_{10}(r)$ [cm]", fontsize=17.5)
ax.set_ylabel(r"$\alpha$", fontsize=17.5)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8),dpi=300)

stp_idx = -1

ax.plot( np.log10(exp_data[stp_idx,1:,0]), exp_data[stp_idx,1:,4] )
ax.set_xlabel(r"$\log_{10}(r)$ [cm]", fontsize=17.5)
ax.set_ylabel(r"$\alpha$", fontsize=17.5)

# Calculate Apparent Horizon Condition

From Alcubierre, eqn. (6.7.15) gives us the expansion H in spherical symmetry as
$$H = \frac{1}{\sqrt{A}} \left( \frac{2}{r} + \frac{\partial_r B}{B} \right) - 2 K^{\theta}_{\theta}$$

With the condition that an apparent horizon forms when $H = 0$.

From the PUSH_GR overleaf, I found $B(r,t) = 1$, $A(r,t) = \Gamma^{-2}$. Further, in `./mathematica/kthetatheta_agile.nb`, I found $K^{\theta}_{\theta}$ and in `./mathematica/kthetatheta_agile.nb` found a simple expression for $H$ with all of these values. It turns out that this was not a differential equation, and so we're able to more easily evaluate H at each timestep and zone of the model.

**However, we found the above didn't yield the behaviour in expansion wrt radius that we expected,** so I rederived it in AGILE coordinates, as described in detail in the PUSH_GR overleaf. I derived this simple relation, which we further found confirmed in Section 4.4.3 of Liebendorfer's thesis:
$$ H = -\frac{2}{r} \left( \Gamma + \frac{\partial_t r}{\alpha} \right),$$
which further reduces to a condition for apparent horizons, $H = 0$, i.e.
$$ \Gamma + u = 0 $$

In [ ]:
def H(data_geo):
    """
    Calculate the expansion H at the given times, and zones, of the star.
    Note that this excludes calculating H at the central zone, as
    Gamma cannot be calculated in the central zone (as r = 0 in this zone
    in AGILE).
    
    **Technically, this isn't actually the expansion, but the 
    apparent horizon condition!** See above markdown cell for the full
    expansion expression.
    
    Parameters 
    ==========
    data_geo: float ndarray (N_stp, N_zones, {hydro variables}) of 
        hydrodynamic data in geometric units, where final index is 
        constructed such that 0 is the radius, 1 is the velocity u,
        and 5 is Gamma (see Gamma calculation function above).
    
    
    Returns
    =======
    float ndarray (N_stp, N_zones - 1): expansion in zones 2 - 180
        (excluding the first zone, labeled 1 in AGILE).
    """    
    N_stp = data_geo.shape[0]
    H = np.zeros(( N_stp, 179 ))

    r = data_geo[:,1:,0]
    gamma = data_geo[:,1:,5]
    velocity = data_geo[:,1:,1]
    
    H[:,:] = gamma + velocity
    return H

In [ ]:
exp_H = H(exp_data_geo)
bh_H  = H(bh_data_geo)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot( exp_data_geo[-1,1:,0], exp_data_geo[-1,1:,5], marker="o", label="Gamma" )
ax.plot( exp_data_geo[-1,1:,0], exp_data_geo[-1,1:,1], marker="o", label="velocity" )

plt.legend()

## Plot H at a single timestep

In [ ]:
def plot_H_timestep(x, H, stp_idx, times, bounce_time, xlabel=None, ylabel="Expansion $H$", ylim=None, model_label=" "):
    t = times[stp_idx]
    t_pb = t - bounce_time
    
    ### Plot formatting
    fig, ax = plt.subplots(figsize=(12,8), dpi=300)
    
    ax.tick_params(labelsize=17.5)
    
    ax.set_xlabel(xlabel, fontsize=17.5)
    ax.set_ylabel(ylabel, fontsize=17.5)
    ax.set_title( "Time = %.5f s, %.5f s post-bounce \n %s" % (times[stp_idx], times[stp_idx] - bounce_time, model_label) )
    
    if ylim is not None:
        ax.set_ylim(ylim)
    
    ### Plotting
    ax.axhline(y=0, linestyle="--", color="black")
    ax.plot( x[stp_idx,:], H[stp_idx,:], marker="o" , markersize=5)
    
    return fig, ax

In [ ]:
# load shock radius info
s15_rt_path = os.path.join("./s15.0_trise400_k4.3", "radii_and_timescales.d")
s22_rt_path = os.path.join("./s22.2_trise400_k4.3", "radii_and_timescales.d")

s15_rt_data = np.genfromtxt(s15_rt_path, skip_header=1)
s22_rt_data = np.genfromtxt(s22_rt_path, skip_header=1)

In [ ]:
# 2, 70, 1000

In [ ]:
stp_idx = 1000
fig, ax = plot_H_timestep( 
    np.log10(exp_data[:,1:,0]), exp_H, 
    stp_idx, 
    exp_times, exp_bounce_time, 
    xlabel=r"$\log_{10}(r)$ [cm]",
    model_label="s15 (exploding)",
    #ylim=(-10,10)
)

# get the idx of the pb-time
t_pb = exp_times[stp_idx] - exp_bounce_time
t_pb_idx = np.abs( s15_rt_data[:,0] - t_pb ).argmin()

shock_radius = s15_rt_data[t_pb_idx,2]

ax.axvline( x=np.log10(shock_radius), linestyle="--", color="orange"  )

In [ ]:
stp_idx = 4000

fig, ax = plot_H_timestep( 
    np.log10(bh_data[:,1:,0]), bh_H, 
    stp_idx, 
    bh_times, bh_bounce_time, 
    xlabel=r"$\log_{10}(r)$ [cm]",
    model_label="s22 (bh)",
    #ylim=(-1E3,1E3)
)

# get the idx of the pb-time
t_pb = bh_times[stp_idx] - bh_bounce_time
t_pb_idx = np.abs( s22_rt_data[:,0] - t_pb ).argmin()

shock_radius = s22_rt_data[t_pb_idx,2]

ax.axvline( x=np.log10(shock_radius), linestyle="--", color="orange"  )